In [1]:
from collections import deque
from collections import namedtuple
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, buffer_size, batch_size):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size: maximum size of buffer
            batch_size: size of each training batch
        """
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])

    def add(self, prev_state, prev_action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(prev_state, prev_action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        return random.sample(self.memory, k=self.batch_size)

    def size(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [2]:
import sys
import time
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, regularizers
from keras.optimizers import Adam

class DQNAgent:
    
    def __init__(self, env):
        self.env = env
        self.state_size = env.observation_space.shape[0]
        self.action_size = env.action_space.n
        self.replay_buffer = ReplayBuffer(buffer_size = 100, batch_size = 64)
        self.target_update_frequency = 20
        self.target_update_counter = 0
        self.gamma = 0.95
        self.initial_epsilon = 1
        self.epsilon = self.initial_epsilon
        self.epsilon_decay_rate = 0.9995
        self.min_epsilon = 0.01
        self.training_scores = []
        
        # main model  # gets trained every step
        self.model = self.build_model()

        # Target model this is what we .predict against every step
        self.target_model = self.build_model()
        self.target_model.set_weights(self.model.get_weights())
        
        
    def build_model(self):
        # Define Neural Network architecture for Deep-Q learning Model here
        pass
    
    
    def reset_episode(self, initial_state):
        """Reset variables for a new episode."""
        # Gradually decrease exploration rate
        self.epsilon *= self.epsilon_decay_rate
        self.epsilon = max(self.epsilon, self.min_epsilon)
        
        self.prev_state = self.preprocess_state(initial_state)
        self.prev_action = np.argmax(self.model.predict(self.prev_state))
        return self.prev_action
    
    def preprocess_state(self, state):
        # Preprocessing code
        pass
    
    
    def reset_exploration(self, epsilon=None):
        """Reset exploration rate used when training."""
        self.epsilon = epsilon if epsilon is not None else self.initial_epsilon
    
    
    def plot_scores(self, scores, rolling_window=100):
        """Plot scores and optional rolling mean using specified window."""
        plt.plot(scores); plt.title("Scores");
        rolling_mean = pd.Series(scores).rolling(rolling_window).mean()
        plt.plot(rolling_mean);
    
    
    def act(self, next_state, reward, done, mode="train", time_delay=None):
        """Pick next action and update weights of the neural network (when mode != 'test')."""
        next_state = self.preprocess_state(next_state)
        if mode == "test": 
            # Test mode: Simply produce an action
            action = np.argmax(self.model.predict(next_state))
            if time_delay != None:        # Adding time delay to watch the agent perform at a little slower pace.
                time.sleep(time_delay)
        else:
            # Exploration vs. exploitation
            do_exploration = np.random.uniform(0, 1) < self.epsilon
            if do_exploration:
                # Pick a random action
                action = np.random.randint(0, self.action_size)
            else:
                # Pick the best action from Q table
                action = np.argmax(self.model.predict(next_state))
            
            # Store the experience in replay memory
            self.replay_buffer.add(self.prev_state, self.prev_action, reward, next_state, done)
            
            if self.replay_buffer.size() > self.replay_buffer.batch_size:
                terminal_state = done        # Determine if the episode has ended.
                minibatch = self.replay_buffer.sample()
                
                # X : states, y : predictions
                X = []
                y = []
                
                for prev_state, prev_action, reward, next_state, done in minibatch:
                    # Setting the target for the model to improve upon
                    if not done:
                        target = reward + (self.gamma * np.max(self.target_model.predict(next_state)))
                    else:
                        target = reward

                    new_q_value = self.model.predict(prev_state)
                    new_q_value[action] = target
                   
                    X.array(prev_state)
                    y.array(new_q_value)
                    
                # Fit on all samples as one batch, log only on terminal state
                self.model.fit(np.vstack(X), np.vstack(y), batch_size=self.replay_buffer.batch_size, 
                               verbose=0, shuffle=False)

                if terminal_state:
                    self.target_update_counter += 1

                # If counter reaches set value, update target network with weights of main network
                if self.target_update_counter > self.target_update_frequency:
                    self.target_model.set_weights(self.model.get_weights())
                    self.target_update_counter = 0
        
        # Roll over current state, action for next step
        self.last_state = next_state
        self.last_action = action
        return action
    
    
    def run(self, num_episodes=20000, mode="train", time_delay=0.01, score_threshold=None, weights_path=None, 
            scores_path=None):
        
        """Run agent in given reinforcement learning environment and return scores."""
        scores = []
        max_score = -np.inf
        min_score = np.inf
        max_avg_score = -np.inf
        avg_score = -np.inf
        for i_episode in range(1, num_episodes+1):
            # Initialize episode
            state = self.env.reset()
            action = self.reset_episode(state)
            total_reward = 0
            done = False

            # Roll out steps until done
            while not done:
                if mode=='test':
                    env.render()
                next_state, reward, done, info = self.env.step(action)
                total_reward += reward
                action = self.act(next_state, reward, done, mode, time_delay)
                env.render()
            # Save final score
            scores.append(total_reward)
            # Print episode stats
            if mode == 'train':
                self.training_done = True
                
                if total_reward > max_score:
                    max_score = total_reward
                    
                if total_reward < min_score:
                    min_score = total_reward
                    
                if len(scores) > 100:
                    avg_score = np.mean(scores[-100:])
                    if avg_score > max_avg_score:
                        max_avg_score = avg_score
                        
                print("\rEpisode {}/{} | Episode Score: {} | Min. Score: {} | Max. Score: {} | Current Avg. Score: {} | Max. Average Score: {} | epsilon: {}"
                      .format(i_episode, num_episodes, total_reward, min_score, max_score, 
                              avg_score, max_avg_score, self.epsilon), end="")
                sys.stdout.flush()
            
            # Terminating loop if the agent achieves reward threshold
            if score_threshold != None and max_avg_score > score_threshold:
                print("\nEnvironment solved after {} episodes".format(i_episode, epsilon))
                break
        
        # Close rendering
        env.close()
        if weights_path != None:
            # Save the model weights
            self.model.save_weights(weights_path)
        
        if scores_path != None:
            logs = {"scores" : scores}
            logs = pd.DataFrame.from_dict(data=logs, orient='index')
            logs.to_csv(scores_path ,index=False)
            
        if mode == "test":
            print("\nScore: ", np.mean(scores))
        else:
            self.training_scores.append(scores)

Using TensorFlow backend.
